# Enel. Granadilla. 4 años. Frq= 10 mins.

In [1]:
import ast, json, random, argparse
import os, sys, datetime, shutil
import statistics, smtplib, ssl, re, string
import datetime, math, pickle
import pdb
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px
#
from plotly import tools
from plotly.subplots import make_subplots
from datetime import date, timedelta

## Carga de datos

In [2]:
fich = './Datos_summary_1923_03102023.xlsx'
sheet_to_df_map = pd.read_excel(fich, sheet_name=None)
sheet_to_df_map.keys()
odat=sheet_to_df_map['Sheet1']
odat

,index,GRLL0222FQLM1_x,GRLL0221dwattout2,GRLL0221dvar,GRLL0221TT_MES_T BP1,GRLL0221TT_MES_P_BP,GRLL0221TOTH2O,GRLL0221csgv,GRLL0221afpcs,GRLL0221CTIM,...,GRLL0220DWATT,GRLL0220DPF,GRLL0220TNH_RPM,GRLL0221TTAAC586,GRLL0222TTAAC586,GRLL-MEDAS-EMI-EMISARIO_TEMP_AMB_UA,GRLL-MEDAS-EMI-EMISARIO_PRESION_UA,GRLL0220DF,GRLL0221DF,GRLL0222DF
0,2019-09-12 00:00:00,0,0.041861,-0.107773,92.843274,1014.562318,836868864,26.309298,0.540243,30.475943,...,0.007275,1,5.6223,0,0,22.973584,1018.351358,0.093198,0.121335,0.116257
1,2019-09-12 00:10:00,0,0.04168,-0.107773,87.139139,1014.407639,836868864,26.309298,0.540243,30.549444,...,0.007275,1,5.611864,0,0,22.872396,1018.349156,0.093198,0.121335,0.116257
2,2019-09-12 00:20:00,0,0.045001,-0.107773,82.557352,1014.265748,836868864,26.309298,0.540243,30.382821,...,0.007275,1,5.606741,0,0,22.960759,1018.253283,0.093198,0.121335,0.116257
3,2019-09-12 00:30:00,0,0.046692,-0.107773,78.61093,1014.047876,836868864,26.309298,0.540243,29.792425,...,0.007275,1,5.616194,0,0,22.844276,1018.334683,0.093198,0.121335,0.116257
4,2019-09-12 00:40:00,0,0.040439,-0.107773,75.229267,1014.158248,836868864,26.309298,0.540243,29.792425,...,0.007275,1,5.618591,0,0,22.698559,1018.345932,0.093198,0.121335,0.116257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210459,2023-09-12 23:10:00,4.024974,49.730271,2.029807,77.414982,1012.959066,1303286850.331875,62.412089,112.667617,27.572483,...,56.830961,0.250464,3000.261968,100,100,24.431391,1017.719256,50.007584,50.023406,50.001535
210460,2023-09-12 23:20:00,4.063264,50.422663,1.590955,77.097834,1012.973495,1303289451.605483,61.932667,111.538281,27.530417,...,57.235486,0.147266,3000.584207,100,100,24.26028,1017.72508,50.009199,50.021606,50.007504
210461,2023-09-12 23:30:00,3.950957,48.17391,1.137557,75.076112,1013.051452,1303291958.556297,61.044968,109.708542,27.623137,...,56.229975,0.044067,3000.396365,100,100,24.265218,1017.786045,50.011282,50.013791,49.997962
210462,2023-09-12 23:40:00,3.910528,47.338156,0.818587,75.513928,1013.09052,1303294384.027383,60.243865,107.340495,27.616187,...,55.522513,-0.059131,2999.971347,100,100,24.49856,1017.863232,50.022529,49.98558,49.96864


In [3]:
fich2 = './server_taglist_cpi.xlsx'
sheet_to_df_map = pd.read_excel(fich2, sheet_name=None)
sheet_to_df_map.keys()
varindx = sheet_to_df_map['server_taglist_cpi']
varindx

,TAG,DESCRIPCION,UNIDAD,GLOBAL_CODE_ID,TIPO DE MEDIDA,SERVIDOR PI,PK,URLAF,RANGOMAX,MODEL,LINKED,PROYECTO_ID,ETIQUETA
0,GRLL0221FQLM1,CAUDAL MASICO COMBUSTIBLE,kg/s,71,CAUDAL,PI-GRAN-S1,1,%5C%5CEMSIPW305%5CCPI%5CCCGT%5CGRANADILLA%5CGR...,10.0,1,GRLL0221CPD;GRLL0221dwattout2;GRLL0221CTDA,4,CAUDAL_COMB
1,GRLL0221dwattout2,POTENCIA ACTIVA,MW,71,POTENCIA,PI-GRAN-S1,2,%5C%5CEMSIPW305%5CCPI%5CCCGT%5CGRANADILLA%5CGR...,100.0,1,NaN,4,MP_POT
2,GRLL0221dvar,POTENCIA REACTIVA,Mvar,71,POTENCIA,PI-GRAN-S1,3,%5C%5CEMSIPW305%5CCPI%5CCCGT%5CGRANADILLA%5CGR...,30.0,1,NaN,4,MP_POT_R
3,GRLL0221TT_MES_T BP1,TEMPERATURA SALIDA CHIMENEA BYPASS,degC,71,TEMPERATURA,PI-GRAN-S1,4,%5C%5CEMSIPW305%5CCPI%5CCCGT%5CGRANADILLA%5CGR...,380.0,1,GRLL0221TT_MES_P_BP;GRLL0221TTXM;GRLL0221WQ,4,TEMP_SAL_CHIMBP
4,GRLL0221TT_MES_P_BP,PRESION SALIDA CHIMENEA BYPASS,bara,71,PRESION,PI-GRAN-S1,5,%5C%5CEMSIPW305%5CCPI%5CCCGT%5CGRANADILLA%5CGR...,1030.0,1,GRLL0221TT_MES_T BP1;GRLL0221TTXM;GRLL0221WQ,4,P_SAL_CHIMBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,GRLL0222DF,FRECUENCIA DEL GENERADOR,Hz,72,FRECUENCIA,PI-GRAN-S1,73,%5C%5CEMSIPW305%5CCPI%5CCCGT%5CGRANADILLA%5CGR...,70.0,0,NaN,4,FR_ALT
73,GRLL0220DF,FRECUENCIA DEL GENERADOR,Hz,76,FRECUENCIA,PI-GRAN-S1,74,%5C%5CEMSIPW305%5CCPI%5CCCGT%5CGRANADILLA%5CGR...,70.0,0,NaN,4,FR_ALT
74,GRLL0221WQ,INYECCION DE AGUA,kg/s,71,CAUDAL,PI-GRAN-S1,75,%5C%5CEMSIPW305%5CCPI%5CCCGT%5CGRANADILLA%5CGR...,10.0,0,NaN,4,FLUJO_AGUA
75,GRLL0222WQ,INYECCION DE AGUA,kg/s,72,CAUDAL,PI-GRAN-S1,76,%5C%5CEMSIPW305%5CCPI%5CCCGT%5CGRANADILLA%5CGR...,10.0,0,NaN,4,FLUJO_AGUA


In [4]:
#
# 71  TG1
# 72  TG2
# 76  CICLO VAPOR
#
varindx['GLOBAL_CODE_ID'].unique()

array([71, 76, 72])

In [5]:
#
# Quitamos valores no numéricos de los datos
#
for k in odat.columns:
    idx = odat[odat[k].apply(lambda x: isinstance(x,str))].index
    if len(idx) > 0:
        print('Var: '+k + ' => ' + str(len(idx)))
        odat = odat.drop(index=idx)
        odat[k] = odat[k].astype(float)
    else:
        if k != 'index':
            odat[k] = odat[k].astype(float)
odat.shape

Var: GRLL0222FQLM1_x => 85
Var: GRLL0221dwattout2 => 1
Var: GRLL0221dvar => 191
Var: GRLL0221TT_MES_T BP1 => 1
Var: GRLL0221TOTH2O => 288
Var: GRLL0221afpcs => 31
Var: GRLL0221CTIM => 1
Var: GRLL0221CTDA => 7
Var: GRLL0221TTRF1 => 378
Var: GRLL0221BBA10GS511ET16R => 8331
Var: GRLL0222dvar => 1025
Var: GRLL0222TT_MES_T BP2 => 5
Var: GRLL0222TOTH2O => 56
Var: GRLL0222csgv => 1
Var: GRLL0222TTXSP1 => 2
Var: GRLL0221LAP50CF001XZ01A => 24
Var: GRLL0220cv1_pos_x => 15
Var: GRLL0220TT_IS => 7
Var: GRLL0222LBA10CF001XZ01A => 17
Var: GRLL0222LBD10CF001XZ01A => 1
Var: GRLL0220LCC01CT004JT01A => 26
Var: GRLL-MEDAS-EMI-EMISARIO_TEMP_AMB_UA => 1248


(198723, 77)

## Consistencia entre variables y TAGs

In [6]:
#
# En varindex hay entradas _x y _y que no sabemos qué hacen pero que no encajan con los datos
# Variables de datos que no están en la tabla de Jerarquía (después de quitar los _x e _y
for j in range(1,len(odat.columns)):
    txtj = odat.columns[j]
    idx  = (varindx['TAG'] == txtj)
    ldx  = len(varindx['TAG'].index[idx])
    if ldx == 0:
        print('Var no encontrada {}'.format(txtj))

Var no encontrada GRLL0222FQLM1_x
Var no encontrada GRLL0222FQLM1_y
Var no encontrada GRLL0220cv1_pos_x
Var no encontrada GRLL0220acv1_pos_x
Var no encontrada GRLL0220cv1_pos_y
Var no encontrada GRLL0220acv1_pos_y


In [7]:
# Variables de la tabla de Jerarquía que no están en los datos.
lkeys = '|'.join(odat.columns[1:])
result = varindx.loc[~varindx['TAG'].str.contains(lkeys, case=False)]
result['TAG']

0                         GRLL0221FQLM1
28                        GRLL0222FQLM1
50                      GRLL0220cv1_pos
51                     GRLL0220acv1_pos
74                           GRLL0221WQ
75                           GRLL0222WQ
76    GRLL-MEDAS-EMI-EMISARIO_HR_AMB_UA
Name: TAG, dtype: object

In [8]:
# 
# Corrección de variables: GRLL-MEDAS-EMI-EMISARIO_HR_AMB_UA
odat = odat.rename(columns={'GRLL-MEDAS-EMI-EMISARIO_PRESION_UA':'GRLL-MEDAS-EMI-EMISARIO_HR_AMB_UA', \
                            'GRLL0222FQLM1_x': 'GRLL0222FQLM1', 'GRLL0220cv1_pos_x':'GRLL0220cv1_pos',\
                            'GRLL0220acv1_pos_x': 'GRLL0220acv1_pos'})

In [9]:
# Variables de la tabla de Jerarquía que no están en los datos.
lkeys = '|'.join(odat.columns[1:])
result = varindx.loc[~varindx['TAG'].str.contains(lkeys, case=False)]
result['TAG']

0                          GRLL0221FQLM1
70    GRLL-MEDAS-EMI-EMISARIO_PRESION_UA
74                            GRLL0221WQ
75                            GRLL0222WQ
Name: TAG, dtype: object

## Segmentación por Sistemas

In [10]:
clean = varindx.loc[varindx['DESCRIPCION'].str.contains('POTENCIA ACTIVA'),['TAG','GLOBAL_CODE_ID']]

In [11]:
dtime = {}
def plus30min(idx,dat):
    k  = dat.index.get_loc(idx)
    k1 = dat.iloc[k:k+4].index
    return(k1)
#
for i in clean.index:
    varj= clean.loc[i,'TAG']
    grp = clean.loc[i,'GLOBAL_CODE_ID']
    toff= odat.loc[odat[varj] < 15,:].index
    if len(toff) > 0:
        tmv = plus30min(toff[0],odat)
    for j in range(1,len(toff)):
        tmvp= plus30min(toff[j],odat)
        tmv = tmv.union(tmvp)
    dtime[grp] = tmv
#

In [12]:
dtime[71]

Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,
            9,
       ...
       206310, 206311, 206312, 206313, 206314, 206315, 206316, 206317, 206318,
       206319],
      dtype='int64', length=20891)

In [13]:
dtime[72]

Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,
            9,
       ...
       200197, 200198, 200199, 200200, 200201, 201451, 201452, 201453, 201454,
       201455],
      dtype='int64', length=22470)

In [14]:
dtime[76]

Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,
            9,
       ...
       200055, 200056, 200057, 200058, 200059, 200060, 200061, 200062, 200063,
       200064],
      dtype='int64', length=14428)

In [15]:
dosunouno = odat.loc[list(set(odat.index) - set(dtime[71].union(dtime[72]).union(dtime[76])))].index
dostodo   = odat.loc[list(set(odat.index) - set(dtime[71].union(dtime[72])))].index
unounocero= odat.loc[list(set(dostodo) - set(dosunouno) )].index
unotodovap= odat.loc[list(set(odat.index) - set(dtime[71]).union(dtime[76]))].index
unocerouno= odat.loc[list(set(unotodovap) - set(dosunouno) )].index
todounocer= odat.loc[list(set(odat.index) - set(dtime[72]).union(dtime[76]))].index
cerounouno= odat.loc[list(set(todounocer) - set(dosunouno) )].index
off       = odat.loc[list(set(dtime[71].union(dtime[72]).union(dtime[76])) )].index

In [16]:
vvs = clean['TAG'].tolist()

In [17]:
odat.loc[off,'LABEL'] = '0+0+0'
odat.loc[dosunouno,'LABEL'] = '1+1+1'
odat.loc[unounocero,'LABEL'] = '1+1+0'
odat.loc[unocerouno,'LABEL'] = '1+0+1'
odat.loc[cerounouno,'LABEL'] = '0+1+1'


/tmp/ipykernel_1788220/2309353663.py:1: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0+0+0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [18]:
odat['LABEL'].value_counts()

LABEL
1+1+1    159550
1+0+1     13597
0+0+0     12374
0+1+1     11145
1+1+0      2057
Name: count, dtype: int64

### Subsistema TG1

In [19]:
#
# 71  TG1
#
nname   = varindx.loc[varindx['GLOBAL_CODE_ID'].isin([71]),['TAG','GLOBAL_CODE_ID','DESCRIPCION','UNIDAD']]
nname['NNORM'] = nname['DESCRIPCION'].str.replace(' ','_')+':'+nname['UNIDAD']
# Nombres explicativos de las varaibles creados
dat_unt = odat.loc[:,['index']+nname['TAG'].tolist()]
dat_unt.columns

KeyError: "['GRLL0221FQLM1', 'GRLL0221WQ'] not in index"